In [2]:
from ris import db2  #library designed for SQL database connection and querying
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
print 'Notebook run: {}'.format(timestamp)
print os.getcwd()
# %load_ext sql 

Notebook run: 2019-11-21 16:27
E:\RIS\Staff Folders\Samuel\Requests\AG\Bike_Date_Ver


In [3]:
cdb = pysqldb.DbConnect(type = 'PG', server = 'DOTDEVRHPGSQL01', database = 'CRASHDATA', quiet = True)
bike_db = pysqldb.DbConnect(type = 'PG', server = 'dotdevrhpgsql01', database = 'bicycle', user = cdb.params['user'],
                           password = cdb.params['password'])


Additional database connection details required:
User name (crashdata):bfeng
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: bfeng 
Connection established 2019-11-21 16:27:59
Database connection (PG) to bicycle on dotdevrhpgsql01 - user: bfeng 
Connection established 2019-11-21 16:28:00


In [4]:
new_table = "%s_bike_network_18d" %datetime.datetime.now().strftime('%Y_%m_%d')

# Archive Old Versions of Bike Network

In [13]:
# for tbl in ["2015_nyc_bicycle_network",           
#             "2016_nyc_bicycle_network"
# #             "2017_01_11_bike_network",
# #              "2017_03_13_bike_network"
# #             "2019_01_09_bike_network"
# #             ,"2019_05_03_bike_network_18d"
#            ]:
#     print("""drop table "{tbl}"
#     """.format(tbl = tbl))

drop table "2015_nyc_bicycle_network"
    
drop table "2016_nyc_bicycle_network"
    


In [141]:
# for tbl in ["2015_nyc_bicycle_network" ,
#             "2016_11_02_bike_network",
#             "2016_nyc_bicycle_network"
#             "2017_01_11_bike_network",
#              "2017_03_13_bike_network"
#             "2019_01_09_bike_network"
#             ,"2019_05_03_bike_network_18d"]:
#     bike_db.query("""create table archive."{tbl}"
#     as select * from "{tbl}"
#     """.format(tbl = tbl))

Query run in 71000 microseconds
Query run in 35000 microseconds


In [11]:
for tbl in [#"2015_nyc_bicycle_network" ,
#             "2016_11_02_bike_network",
#             "2016_nyc_bicycle_network"
            "2017_01_11_bike_network",
             "2017_03_13_bike_network",
            "2019_01_09_bike_network",
            "2019_05_03_bike_network_18d"]:
    bike_db.query("""grant all on archive."{tbl}" to public
    """.format(tbl = tbl))

Query run in 1000 microseconds
Query run in 1000 microseconds
Query run in 1000 microseconds
Query run in 1000 microseconds


# Install Date Mismatch

In [89]:
# instdate_disagree = cdb.dfquery("""
                    
#                     select n.bikeid, n.instdate new_instdate, n.segmentid::int new_segmentid,
#                     o.instdate old_instdate,  n.geom from working.current_network n
#                     join working.current_network2 o 
#                     on n.segmentid::int = o.segmentid::int
#                     where n.instdate != o.instdate
#                     and extract(year from o.instdate) < 2017
#                     and extract(year from n.instdate) < 2017
#                     and extract(year from o.moddate) < 2017
#                     and n.segmentid != '0' 
#                     order by n.segmentid::int

                    
#                     """)
   

In [90]:
# instdate_disagree.to_csv("install_mismatch.csv",index=False)

# Old Moddate > New Moddate

In [91]:
# moddate_disagree = cdb.dfquery("""
#                     select n.bikeid, n.moddate new_moddate, n.segmentid::int new_segmentid,
#                     o.moddate old_moddate, n.geom from working.current_network n
#                     join working.current_network2 o 
#                     on n.segmentid::int = o.segmentid::int
#                     where o.moddate > n.moddate
#                     and extract(year from o.instdate) < 2017
#                     and extract(year from n.instdate) < 2017
#                     and extract(year from o.moddate) < 2017
#                     and n.segmentid != '0' 
#                     order by n.segmentid::int""")

# Lanes in 2016 network not in 2018 network

In [92]:
# missing_lanes = cdb.dfquery("""

#                     select distinct o2.* from (
#                     select o.instdate old_instdate, o.moddate old_moddate, o.segmentid old_segmentid, o.geom old_geom, o.bikeid
#                     from working.proximity_bike_facility n
#                     right join working.proximity_bike_facility2 o 
#                     on n.segmentid::int = o.segmentid::int
#                     where n.segmentid is null
#                     and o.segmentid not in ('99999999', '0000000')
#                     and extract(year from o.moddate) < 2017
#                     order by n.segmentid::int
#                     ) o2
#                     left join working.proximity_bike_facility n2
#                     on st_within(o2.old_geom, st_buffer(n2.geom, 25))
#                     where n2.geom is null
#                     and o2.bikeid not in ('779', '767')


                    
#                     """)
   

In [93]:
# missing_lanes.to_csv("missing_lanes.csv",index=False)

# Install Date > Mod Date

In [94]:
# instmod = cdb.dfquery("""select * from working.current_network
# where instdate > moddate
# and bikeid::float != 1166""")

# 18 Bike Network Segmentid = 0

In [95]:
# seg18_0 = cdb.dfquery("""
# select n.*, l.segmentid real_segmentid, st_intersection(st_buffer(n.geom, 1), l.geom)
# , st_length(st_intersection(st_buffer(n.geom, 1), l.geom))/st_length(n.geom) 
# from working.current_network n
# join lion l
# on st_dwithin(n.geom, l.geom, 10)
# where n.segmentid = '0'
# and st_length(st_intersection(st_buffer(n.geom, 1), l.geom))/st_length(n.geom) > 0.8
# """)

# 16 Bike Network Segmentid = 0/No equivalent in 18d

In [96]:
# seg16_0 = cdb.dfquery("""
# select n.*, l.segmentid real_segmentid, l.geom geom_18, st_intersection(st_buffer(n.geom, 1), l.geom)
# , st_length(st_intersection(st_buffer(n.geom, 10), l.geom))/st_length(n.geom)
# from working.current_network2 n
# join working.current_network l
# on st_dwithin(n.geom, l.geom, 10)
# where n.segmentid in ('99999999', '0000000')
# and st_length(st_intersection(st_buffer(n.geom, 1), l.geom))/st_length(l.geom) > 0.8;
# """)

# insert lion segment ped bridge

In [97]:
bike_db.query("""drop table if exists "{table}";

create table "{table}" as 
select * from "2019_05_03_bike_network_18d"
union 
select 
gid::numeric,
null::numeric objectid,
__gid::numeric,
segmentid::numeric,
segmentmil::numeric,
lanemiles::numeric,
bikeid::numeric,
shape_leng::numeric,
objectid_1::numeric,
street::varchar(254),
boro::numeric,
facilitycl::varchar(254),
fromstreet::varchar(254),
tostreet::varchar(254),
lasteditby::varchar(254),
onoffst::varchar(254),
allclasses::varchar(254),
instdate::date,
moddate::date,
comments::varchar(254),
bikedir::varchar(254),
lanecount::numeric,
jurisdicti::varchar(254),
ft_facilit::varchar(254),
tf_facilit::varchar(254),
lastedited::date,
sip_name::varchar(100),
prev_class::varchar(50),
prev_facil::varchar(50),
null::double precision bikeid_1,
null::integer priority_d,
null::date pg_upload,
null::integer sip_year,
geom::geometry
from "2017_03_13_bike_network" 
where bikeid::int = 1230 or segmentid::int in (238429, 238435)
""".format(table = new_table))

Query run in 132000 microseconds


In [98]:
bike_db.query("""insert into "{table}" (segmentid, bikeid_1, geom)
select segmentid::int, 2389, geom from c_lion_18d
where segmentid::int = 312394;

update "{table}"
set bikeid = b.bikeid,
objectid_1 = b.objectid_1,
street = b.street,
boro = b.boro,
facilitycl = b.facilitycl,
fromstreet = b.fromstreet,
tostreet = b.tostreet,
onoffst = b.onoffst,
allclasses = b.allclasses,
instdate = b.instdate,
moddate = b.moddate,
bikedir = b.bikedir,
lanecount = b.lanecount,
jurisdicti = b.jurisdicti,
bikeid_1 = b.bikeid_1,
priority_d = b.priority_d,
pg_upload = b.pg_upload,
sip_year = b.sip_year
from 
(select bikeid, objectid_1, street, boro, facilitycl, fromstreet, tostreet,
onoffst, allclasses, instdate, moddate, bikedir, lanecount, jurisdicti, bikeid_1,
priority_d, pg_upload, sip_year
from "{table}" where bikeid = 2389) b
where segmentid = 312394

""".format(table=new_table))

Query run in 132000 microseconds


In [99]:
data = pd.read_csv('install_mismatch.csv')

In [100]:
changes = data.loc[data['comment'].str.startswith('old')]

In [101]:
change_dict = {}
for i in range(len(changes)):
    change_dict[changes.iloc[i].bikeid] = changes.iloc[i].old_instdate

In [102]:
# bld changes
change_dict

{32: '7/22/2011', 544: '11/15/2011', 1510: '7/1/2010', 2652: '7/1/2010'}

In [103]:
bike_db.query("""alter table "{table}"
add column edited text""".format(table = new_table))

Query run in 3000 microseconds


In [104]:
for k in change_dict.keys():
    bike_db.query("""update "{table}"
    set instdate = '{new_date}'
    where bikeid::int = {k};
    
    update "{table}"
    set edited = 'instdate updated'
    where bikeid::int = {k};
    
    update "{table}"
    set edited = 'instdate, moddate updated'
    where bikeid::int = {k}
    and instdate > moddate;
    
    update "{table}"
    set moddate = '{new_date}'
    where bikeid::int = {k}
    and instdate > moddate;
    """.format(table = new_table, new_date = change_dict[k], k = k))

Query run in 26000 microseconds
Query run in 25000 microseconds
Query run in 25000 microseconds
Query run in 24000 microseconds


In [105]:
test = bike_db.dfquery("""
select n.*, l.segmentid real_segmentid
from "2019_05_03_bike_network_18d" n
join c_lion_18d l
on st_dwithin(n.geom, l.geom, 10)
where n.segmentid = '0'
and st_length(st_intersection(st_buffer(n.geom, 1), l.geom))/st_length(n.geom) > 0.8
""")

In [106]:
change_dict2 = {}
for i in range(len(test)):
    change_dict2[test.iloc[i].objectid] = test.iloc[i].real_segmentid

In [107]:
change_dict2

{Decimal('3960'): '0234018',
 Decimal('12429'): '0004871',
 Decimal('15744'): '0256635',
 Decimal('15745'): '0256645',
 Decimal('15746'): '0256644',
 Decimal('15747'): '0256636',
 Decimal('16222'): '0293222',
 Decimal('16676'): '0161364',
 Decimal('16677'): '0161365',
 Decimal('16679'): '0161361',
 Decimal('16685'): '0161360',
 Decimal('16689'): '0132715',
 Decimal('16690'): '0161336',
 Decimal('16691'): '0161335',
 Decimal('16692'): '0161337'}

In [108]:
for k in change_dict2.keys():
    bike_db.query("""
    update "{table}"
    set edited = 'segmentid updated'
    where objectid = {k};
    
    update "{table}"
    set segmentid = '{new_segmentid}'
    where objectid = {k};
    
    """.format(table = new_table, new_segmentid = change_dict2[k], k = k))

Query run in 17000 microseconds
Query run in 16000 microseconds
Query run in 10000 microseconds
Query run in 10000 microseconds
Query run in 12000 microseconds
Query run in 11000 microseconds
Query run in 16000 microseconds
Query run in 10000 microseconds
Query run in 10000 microseconds
Query run in 12000 microseconds
Query run in 13000 microseconds
Query run in 13000 microseconds
Query run in 10000 microseconds
Query run in 11000 microseconds
Query run in 13000 microseconds


# Manual Reviews Confirmed by Bikes

In [109]:
# im = cdb.dfquery("""select * from working.install_mismatch""")

In [110]:
# im.bikeid.unique()

In [111]:
change_dict3 = {2670:'2016-08-26', 2671:'2016-08-26', 544: '2011-11-15'}
seg_dict = {239994:'350', 239999:'350'}
# insert 312394 segment as part of 2389 bikeid

In [112]:
for k in change_dict3.keys():
    bike_db.query("""update "{table}"
    set instdate = '{new_date}'
    where bikeid::int = {k};
    
    update "{table}"
    set edited = 'instdate updated'
    where bikeid::int = {k};
    """.format(table = new_table, new_date = change_dict3[k], k = str(k)))

Query run in 18000 microseconds
Query run in 16000 microseconds
Query run in 17000 microseconds


In [113]:
for k in seg_dict.keys():
    bike_db.query("""update "{table}"
    set bikeid = '{new_id}'
    where segmentid::int = {k};
    
    update "{table}"
    set bikeid_1 = '{new_id}'
    where segmentid::int = {k};
    
    update "{table}"
    set fromstreet = 'E 125 ST'
    where segmentid::int = {k};
        
    update "{table}"
    set tostreet = 'E 60 ST'
    where segmentid::int = {k};
    
    update "{table}"
    set street = 'FDR DR GREENWAY'
    where segmentid::int = {k};

    update "{table}"
    set ft_facilit = 'Greenway'
    where segmentid::int = {k};
    
    update "{table}"
    set tf_facilit = 'Greenway'
    where segmentid::int = {k};
    
    update "{table}"
    set edited = 'bikeid, streets, facility type updated'
    where segmentid::int = {k};
    
    update "{table}"
    set instdate = '1939-01-01'
    where segmentid::int = {k};   
    
    update "{table}"
    set moddate = '1939-01-01'
    where segmentid::int = {k};
    
    """.format(table = new_table, new_id = seg_dict[k], k = str(k)))

Query run in 40000 microseconds
Query run in 40000 microseconds


In [114]:
# om = cdb.dfquery("""select * from working.old_new_moddate""")

In [115]:
# om.bikeid.unique()

In [116]:
# add comment to say segments fully completed in 2013-11-05 for segment 312556
change_dict4 = {32: '2011-07-22'}

In [117]:
for k in change_dict4.keys():
    bike_db.query("""update "{table}"
    set instdate = '{new_date}'
    where bikeid::int = {k};
    
    update "{table}"
    set moddate = '{new_date}'
    where bikeid::int = {k};
    
    update "{table}"
    set edited = 'instdate, moddate updated'
    where bikeid::int = {k};
    """.format(table = new_table, new_date = change_dict4[k], k = str(k)))

Query run in 22000 microseconds


In [118]:
# # insert these
# ml = cdb.dfquery("""select * from working.missing_lanes where bikeid::int = 1230 or old_segmentid::int in (238429, 238435)""")

In [119]:
# imod = cdb.dfquery("""select * from working.inst_mod""")
# imod.bikeid.unique()

In [120]:
change_dict5 = {91:'2008-11-17', 94:'2008-11-17', 96:'2008-11-17', 2483:'2017-10-06'}

In [121]:
for k in change_dict5.keys():
    bike_db.query("""update "{table}"
    set instdate = '{new_date}'
    where bikeid::int = {k};
    
    update "{table}"
    set moddate = '{new_date}'
    where bikeid::int = {k};
    
    update "{table}"
    set edited = 'instdate, moddate updated'
    where bikeid::int = {k};
    """.format(table = new_table, new_date = change_dict5[k], k = str(k)))

Query run in 20000 microseconds
Query run in 21000 microseconds
Query run in 17000 microseconds
Query run in 15000 microseconds


In [122]:
change_dict6 = {2640:'2018-09-21'}
for k in change_dict5.keys():
    bike_db.query("""
    update "{table}"
    set moddate = '{new_date}'
    where bikeid::int = {k};
    
    update "{table}"
    set edited = 'moddate updated'
    where bikeid::int = {k};
    """.format(table = new_table, new_date = change_dict5[k], k = str(k)))

Query run in 12000 microseconds
Query run in 17000 microseconds
Query run in 11000 microseconds
Query run in 14000 microseconds


In [123]:
bike_db.query("""update "{table}"
set comments = 'Due to lion version changes, segment had some portion finished on 11/5/13. Original segment had fully completed lane corresponding to install date.'
where segmentid::int = 312556
""".format(table = new_table))

Query run in 12000 microseconds


In [124]:
bike_db.dfquery("""select * from "{table}" where segmentid in (29150, 312394)""".format(table = new_table))

,gid,objectid,__gid,segmentid,segmentmil,lanemiles,bikeid,shape_leng,objectid_1,street,...,lastedited,sip_name,prev_class,prev_facil,bikeid_1,priority_d,pg_upload,sip_year,geom,edited
0,None,None,None,312394,None,None,2389.00000000,None,2381,E 78 ST OVERPASS,...,None,None,None,None,2389.0,0,2018-01-29,0,0105000020D70800000100000001020000000300000000...,None
1,9581,9581,1615,29150,0.0390221041530,0.0390221041530,1510.00000000,206.036661294,1502,VANDERBILT AV,...,2018-10-19,"Vanderbilt Av, Clermont Av",III,Sharrows,1510.0,0,2018-01-29,2018,0105000020D70800000100000001020000000200000080...,instdate updated


In [125]:
bike_db.shp_to_table(path = "E:\RIS\Staff Folders\Bryant\Bikes\For Michael", table = 'updates_18', 
                    schema = 'public', shp_name = 'to_be_changed.shp')

Query run in 14000 microseconds
Query run in 2000 microseconds
Query run in 2000 microseconds
Query run in 9000 microseconds
Query run in 2000 microseconds
Query run in 2000 microseconds


In [126]:
bike_db.query("""delete from "{table}" where segmentid in (29150, 312394);

alter table updates_18
drop ogc_fid;

alter table updates_18
drop _uid_;

drop table if exists bike_network_{d}; 
create table bike_network_{d} as 
select * from "{table}"
union
select gid,
  objectid,
  __gid,
  segmentid,
  segmentmil,
  lanemiles,
  bikeid,
  shape_leng,
  objectid_1,
  street,
  boro,
  facilitycl,
  fromstreet,
  tostreet,
  lasteditby,
  onoffst,
  allclasses,
  instdate,
  moddate,
  comments,
  bikedir,
  lanecount,
  jurisdicti,
  ft_facilit,
  tf_facilit,
  lastedited,
  sip_name,
  prev_class,
  prev_facil,
  bikeid_1,
  priority_d,
  pg_upload,
  sip_year,
  geom,
  edited from updates_18
""".format(table = new_table, d = datetime.datetime.now().strftime('%Y_%m_%d'))
)

Query run in 195000 microseconds


In [127]:
bike_db.connect()

Database connection (PG) to bicycle on dotdevrhpgsql01 - user: bfeng 
Connection established 2019-11-21 10:32:28


In [129]:
bike_db.query_to_shp("""select * from "bike_network_%s" """ % datetime.datetime.now().strftime('%Y_%m_%d'),
                     path = 'E:\RIS\Staff Folders\Bryant\Bikes\For Michael',
                    shp_name = 'updated_18_network.shp')

Query run in 155000 microseconds
updated_18_network.shp shapefile 
written to: E:\RIS\Staff Folders\Bryant\Bikes\For Michael
generated from: select * from "bike_network_2019_11_21" 


In [133]:
bike_db.query("""drop table if exists "{table}" """.format(table = new_table))

Query run in 3000 microseconds


# For Bikes to Check

In [114]:
# install date disagree
check = data.loc[~data['comment'].str.startswith('old')].bikeid.tolist()

In [132]:
set(check)

{2287L, 2389L, 2670L, 2671L, 2731L}

In [4]:
cdb.query("""drop table if exists working.install_mismatch;
create table working.install_mismatch as
select n.bikeid, n.instdate new_instdate, n.segmentid::int new_segmentid,
                    o.instdate old_instdate,  n.geom from working.current_network n
                    join working.current_network2 o 
                    on n.segmentid::int = o.segmentid::int
                    where n.instdate != o.instdate
                    and extract(year from o.instdate) < 2017
                    and extract(year from n.instdate) < 2017
                    and extract(year from o.moddate) < 2017
                    and n.segmentid != '0' 
                    and n.bikeid::float in (2287, 2389, 2670, 2671, 2731)
                    order by n.segmentid::int""")

Query run in 29000 microseconds


In [128]:
check2

['32.0000000000',
 '32.0000000000',
 '32.0000000000',
 '32.0000000000',
 '2640.00000000',
 '2640.00000000',
 '2640.00000000',
 '2640.00000000',
 '32.0000000000',
 '32.0000000000',
 '32.0000000000',
 '32.0000000000',
 '1091.00000000']

In [127]:
# old moddate > new moddate
check2 = moddate_disagree.bikeid.tolist()

In [9]:
cdb.query("""drop table if exists working.old_new_moddate;
create table working.old_new_moddate as
select n.bikeid, n.moddate new_moddate, n.segmentid::int new_segmentid,
                    o.moddate old_moddate, n.geom from working.current_network n
                    join working.current_network2 o 
                    on n.segmentid::int = o.segmentid::int
                    where o.moddate > n.moddate
                    and extract(year from o.instdate) < 2017
                    and extract(year from n.instdate) < 2017
                    and extract(year from o.moddate) < 2017
                    and n.segmentid != '0' 
                    order by n.segmentid::int;

""")

Query run in 37000 microseconds


In [10]:
# 16 bike lanes not in 18
data2 = pd.read_csv('bike_lane_data2_bf.csv')
check3 = data2.loc[~data2['Unnamed: 7'].isin(['removed','moved', 'lion verion difference'])].bikeid.tolist()

In [12]:
check3

[1230L, 1230L, 1230L, 1071L, 2034L, 2034L, 2034L, 2034L, 407L, 407L]

In [15]:
cdb.query("""drop table if exists working.missing_lanes;
                create table working.missing_lanes as
                    select distinct o2.* from (
                    select o.instdate old_instdate, o.moddate old_moddate, o.segmentid old_segmentid, o.geom old_geom, o.bikeid
                    from working.current_network n
                    right join working.current_network2 o 
                    on n.segmentid::int = o.segmentid::int
                    where n.segmentid is null
                    and o.segmentid not in ('99999999', '0000000')
                    and extract(year from o.moddate) < 2017
                    order by n.segmentid::int
                    ) o2
                    left join working.current_network n2
                    on st_within(o2.old_geom, st_buffer(n2.geom, 25))
                    where n2.geom is null
                    and o2.bikeid not in ('779', '767')
                    and o2.bikeid in ('1230', '1071', '2034','407')
                    """)

Query run in 8 seconds


In [131]:
# instdate > moddate
check4 = instmod.bikeid.tolist()

In [121]:
check4

['91.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '94.0000000000',
 '96.0000000000',
 '96.0000000000',
 '96.0000000000',
 '96.0000000000',
 '2483.00000000',
 '2640.00000000',
 '2640.00000000',
 '2640.00000000',
 '2640.00000000']

In [16]:
instmod = cdb.query("""drop table if exists working.inst_mod;
create table working.inst_mod as
select * from working.current_network
where instdate > moddate
and bikeid::float != 1166""")

Query run in 14000 microseconds


In [21]:
check5 = pd.read_excel('Bike_Lane_Check.xlsx')

In [27]:
check5.loc[~check5.Status.isin(['Delete', 'Exist in 18D but TBD'])]

,16D Segment,16D Street Name,Status,18D Segments,Unnamed: 4,Unnamed: 5,16N Bike ID,Status.1
3,275387,JORALEMON STREET,Switch to 18D Segment,316415.0,NaN,NaN,817.0,Exist in 18N
4,275332,JORALEMON STREET,Switch to 18D Segment,316415.0,NaN,NaN,NaN,NaN
5,225299,EAST 1 STREET,Switch to 18D Segment,311349.0,not real street,NaN,NaN,NaN
12,865,VAN CORTLANDT PARK GREENWAY,TBD,NaN,boundaries in 18d,NaN,NaN,NaN


In [32]:
cdb.query("""drop table if exists working.version_diff;
create table working.version_diff as 
select *
from working.current_network2 n
where n.segmentid in ('99999999', '0000000')
and n.bikeid in ('234','865','1248');
""")

Failure:

- Query run 2019-09-27 12:00:17.903000
	drop table if exists working.version_diff;
create table working.version_diff as 
select *
from working.current_network2 n
where n.segmentid in ('99999999', '0000000')
and n.bikeid in ('234','865','1248');



SystemExit: 